<a href="https://colab.research.google.com/github/yakubszatkowski/100_days_python/blob/master/push/Google_Play_Store_App_Analytics_(start).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [5]:
df_apps = pd.read_csv('drive/MyDrive/Colab Notebooks/Day 75 - Analysing the Android App Store/apps.csv')

# Notebook Presentation

In [6]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [7]:
df_apps.shape

(10841, 12)

In [8]:
df_apps.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')

In [9]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
2467,Safe Wi-Fi,PRODUCTIVITY,2.90,32,9.70,"1,000",Free,0,Everyone,Productivity,"August 3, 2018",4.4 and up
3094,CT POSITIONING,MEDICAL,4.50,18,9.30,"5,000",Free,0,Everyone,Medical,"March 10, 2018",4.0.3 and up
8704,"Shutterfly: Free Prints, Photo Books, Cards, G...",PHOTOGRAPHY,4.60,98716,59.00,"5,000,000",Free,0,Everyone,Photography,"August 1, 2018",5.0 and up
5064,Basketball FRVR - Shoot the Hoop and Slam Dunk!,FAMILY,4.50,4076,19.00,"100,000",Free,0,Everyone,Sports;Action & Adventure,"August 1, 2018",4.1 and up
2612,BW-Go,GAME,4.80,265,1.30,"1,000",Paid,$3.49,Everyone,Board,"June 18, 2015",2.3 and up


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns. 

In [10]:
df_apps.drop(['Last_Updated', 'Android_Ver'], inplace=True, axis=1)
df_apps.head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
0,Ak Parti Yardım Toplama,SOCIAL,NaN,0,8.70,0,Paid,$13.99,Teen,Social
1,Ain Arabic Kids Alif Ba ta,FAMILY,NaN,0,33.00,0,Paid,$2.99,Everyone,Education
2,Popsicle Launcher for Android P 9.0 launcher,PERSONALIZATION,NaN,0,5.50,0,Paid,$1.49,Everyone,Personalization
3,Command & Conquer: Rivals,FAMILY,NaN,0,19.00,0,NaN,0,Everyone 10+,Strategy
4,CX Network,BUSINESS,NaN,0,10.00,0,Free,0,Everyone,Business


### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows. 

In [11]:
# df_apps.isna().sum()
df_apps.isna().values.sum()

1475

In [12]:
# different method
nan_rows = df_apps[df_apps.Rating.isna()] # this shows rows with N/A values for rating column
nan_rows.shape

(1474, 10)

In [13]:
df_apps_clean = df_apps.dropna()
df_apps_clean.shape

(9367, 10)

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`. 


In [14]:
df_apps_clean.duplicated().sum()  # amount of duplicates

476

In [15]:
df_apps_clean[df_apps_clean.duplicated()]  # duplicated rows

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
946,420 BZ Budeze Delivery,MEDICAL,5.00,2,11.00,100,Free,0,Mature 17+,Medical
1133,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating
1196,"Cardiac diagnosis (heart rate, arrhythmia)",MEDICAL,4.40,8,6.50,100,Paid,$12.99,Everyone,Medical
1231,Sway Medical,MEDICAL,5.00,3,22.00,100,Free,0,Everyone,Medical
1247,Chat Kids - Chat Room For Kids,DATING,4.70,6,4.90,100,Free,0,Mature 17+,Dating
...,...,...,...,...,...,...,...,...,...,...
10802,Skype - free IM & video calls,COMMUNICATION,4.10,10484169,3.50,"1,000,000,000",Free,0,Everyone,Communication
10809,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10826,Google Drive,PRODUCTIVITY,4.40,2731211,4.00,"1,000,000,000",Free,0,Everyone,Productivity
10832,Google News,NEWS_AND_MAGAZINES,3.90,877635,13.00,"1,000,000,000",Free,0,Teen,News & Magazines


In [16]:
# My take on how to count rows which contain word 'Instagram' in App column
df_apps_clean.App.str.contains('Instagram').sum()
# Angela's take on how to find rows with 'Instagram' in App column
df_apps_clean[df_apps_clean.App == 'Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10808,Instagram,SOCIAL,4.50,66577446,5.30,"1,000,000,000",Free,0,Teen,Social
10809,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.50,66509917,5.30,"1,000,000,000",Free,0,Teen,Social


In [17]:
# This method wont work because it contains different amount of reviews
df_apps_clean_wodupes = df_apps_clean.drop_duplicates()
df_apps_clean_wodupes[df_apps_clean_wodupes.App == 'Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10808,Instagram,SOCIAL,4.50,66577446,5.30,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.50,66509917,5.30,"1,000,000,000",Free,0,Teen,Social


In [18]:
# We need to specify the subset for indentifying dupelicates
df_apps_clean_wodupes = df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'])  # Should be able to do the same with just putting in 'App', however amount of rows slightly differs
df_apps_clean_wodupes[df_apps_clean_wodupes.App == 'Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social


In [19]:
df_apps_clean_wodupes.shape

(8199, 10)

In [20]:
# RESET BUTTON
# df_apps_clean_wodupes = df_apps_clean

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [21]:
df_apps_clean_wodupes.sort_values('Rating', ascending=False).head(10)
#alternative way
df_apps_clean_wodupes.nlargest(10, 'Rating')

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,$1.49,Everyone,Arcade
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,$0.99,Everyone,Arcade
82,Brick Breaker BR,GAME,5.00,7,19.00,5,Free,0,Everyone,Arcade
99,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.00,1,4.60,5,Free,0,Everyone,Medical
126,Tablet Reminder,MEDICAL,5.00,4,2.50,5,Free,0,Everyone,Medical
128,CQ ESPM,BUSINESS,5.00,2,3.40,5,Free,0,Everyone,Business
141,Clinic Doctor EHr,MEDICAL,5.00,2,7.10,5,Free,0,Everyone,Medical
151,EB Cash Collections,BUSINESS,5.00,1,4.30,5,Free,0,Everyone,Business
154,CZ-Help,BOOKS_AND_REFERENCE,5.00,2,1.40,5,Free,0,Everyone,Books & Reference


The problem is the best rating stats have usually small amount of rewiews, in order to truly gather relevant date we would have to take amount of reviews into account.

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please? 

In [22]:
df_apps_clean_wodupes.nlargest(5, 'Size_MBs')

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
1795,Navi Radiography Pro,MEDICAL,4.70,11,100.00,500,Paid,$15.99,Everyone,Medical
3144,Vi Trainer,HEALTH_AND_FITNESS,3.60,124,100.00,"5,000",Free,0,Everyone,Health & Fitness
4176,Car Crash III Beam DH Real Damage Simulator 2018,GAME,3.60,151,100.00,"10,000",Free,0,Everyone,Racing
7926,Post Bank,FINANCE,4.50,60449,100.00,"1,000,000",Free,0,Everyone,Finance
7927,The Walking Dead: Our World,GAME,4.00,22435,100.00,"1,000,000",Free,0,Teen,Action


# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [23]:
df_apps_clean_wodupes.nlargest(5, 'Reviews')

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,"1,000,000,000",Free,0,Everyone,Communication
10650,Clash of Clans,GAME,4.60,44891723,98.00,"100,000,000",Free,0,Everyone 10+,Strategy


In [24]:
most_popular_apps = df_apps_clean_wodupes.nlargest(50, 'Reviews')

if most_popular_apps[most_popular_apps['Type'] == 'Paid'].empty:
  print('None of the 50 most popular applications are Paid')

None of the 50 most popular applications are Paid


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings
Different ways of how to present pie chart

In [25]:
ratings = df_apps_clean_wodupes.Content_Rating.value_counts()
ratings

Everyone           6621
Teen                912
Mature 17+          357
Everyone 10+        305
Adults only 18+       3
Unrated               1
Name: Content_Rating, dtype: int64

In [26]:
fig = px.pie(labels=ratings.index, values=ratings.values)
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:137: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  return args["labels"][column]


In [27]:
fig = px.pie(labels=ratings.index, values=ratings.values, names=ratings.index, title='Content Rating')
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



In [28]:
fig = px.pie(labels=ratings.index, values=ratings.values, names=ratings.index, title='Content Rating')
fig.update_traces(textposition='inside', textinfo='percent', textfont_size=15)
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install? 

Check the datatype of the Installs column.

Count the number of apps at each level of installations. 

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first. 

In [29]:
type(df_apps_clean_wodupes.Installs[10824])

str

In [30]:
# df_apps_clean_wodupes.Installs.info()
# or
df_apps_clean_wodupes.Installs.describe()

count          8199
unique           19
top       1,000,000
freq           1417
Name: Installs, dtype: object

In [31]:
# installation_amounts = df_apps_clean_wodupes.Installs.value_counts()
# or 
# installation_amounts = df_apps_clean_wodupes.groupby('Installs').count().App  # can also add .to_frame()
# or
installation_amounts = df_apps_clean_wodupes[['App', 'Installs']].groupby('Installs').count()
installation_amounts

,App
Installs,
1,3
"1,000",698
"1,000,000",1417
"1,000,000,000",20
10,69
"10,000",988
"10,000,000",933
100,303
"100,000",1096


In [32]:
# My take on converting istallation 'object' into 'int' - first it gets rid of commas then it changes type to int
# installations_df = df_apps_clean_wodupes.Installs.replace(',', '', regex=True).astype('int')
# or
installations_df = df_apps_clean_wodupes.Installs.replace(',', '', regex=True)
installations_df = pd.to_numeric(installations_df)
installations_df 

21                1
28                1
47                1
82                5
99                5
            ...    
10824    1000000000
10828    1000000000
10829    1000000000
10831    1000000000
10835    1000000000
Name: Installs, Length: 8199, dtype: int64

In [33]:
# finally 
df_apps_numinstalls = df_apps_clean_wodupes
df_apps_numinstalls.Installs = installations_df
df_apps_numinstalls[['Installs', 'App']].groupby('Installs').count()

<ipython-input-33-7c97471d131d>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,App
Installs,
1,3
5,9
10,69
50,56
100,303
500,199
1000,698
5000,425
10000,988


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [34]:
df_price = df_apps_numinstalls
df_price.Price = df_price.Price.str.replace('$', '', regex=True)
df_price.Price = df_price.Price.astype(float)  # alt: df_price.Price = pd.to_numeric(df_price.Price)
df_price.Price

<ipython-input-34-39028dc23d5a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-34-39028dc23d5a>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



21      0.00
28      1.49
47      0.99
82      0.00
99      0.00
        ... 
10824   0.00
10828   0.00
10829   0.00
10831   0.00
10835   0.00
Name: Price, Length: 8199, dtype: float64

In [35]:
most_expensive = df_price.nlargest(20, 'Price')
most_expensive

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
3946,I'm Rich - Trump Edition,LIFESTYLE,3.60,275,7.30,10000,Paid,400.00,Everyone,Lifestyle
1331,most expensive app (H),FAMILY,4.30,6,1.50,100,Paid,399.99,Everyone,Entertainment
1946,I am rich (Most expensive app),FINANCE,4.10,129,2.70,1000,Paid,399.99,Teen,Finance
2394,I am Rich!,FINANCE,3.80,93,22.00,1000,Paid,399.99,Everyone,Finance
2461,I AM RICH PRO PLUS,FINANCE,4.00,36,41.00,1000,Paid,399.99,Everyone,Finance
2775,I Am Rich Pro,FAMILY,4.40,201,2.70,5000,Paid,399.99,Everyone,Entertainment
3114,I am Rich,FINANCE,4.30,180,3.80,5000,Paid,399.99,Everyone,Finance
3145,I am rich(premium),FINANCE,3.50,472,0.94,5000,Paid,399.99,Everyone,Finance
3221,I am Rich Plus,FAMILY,4.00,856,8.70,10000,Paid,399.99,Everyone,Entertainment
3554,💎 I'm rich,LIFESTYLE,3.80,718,26.00,10000,Paid,399.99,Everyone,Lifestyle


### The most expensive apps sub $250

In [36]:
ids_to_delete = df_price[df_price.Price > 250].index  # selecting the apps with the price over 250
df_price = df_price.drop(ids_to_delete)  # i forgot to assign back to df_price here
df_price.nlargest(10, 'Price')

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
2281,Vargo Anesthesia Mega App,MEDICAL,4.60,92,32.00,1000,Paid,79.99,Everyone,Medical
1407,LTC AS Legal,MEDICAL,4.00,6,1.30,100,Paid,39.99,Everyone,Medical
2629,I am Rich Person,LIFESTYLE,4.20,134,1.80,1000,Paid,37.99,Everyone,Lifestyle
2481,A Manual of Acupuncture,MEDICAL,3.50,214,68.00,1000,Paid,33.99,Everyone,Medical
504,AP Art History Flashcards,FAMILY,5.00,1,96.00,10,Paid,29.99,Mature 17+,Education
2207,EMT PASS,MEDICAL,3.40,51,2.40,1000,Paid,29.99,Everyone,Medical
2463,PTA Content Master,MEDICAL,4.20,64,41.00,1000,Paid,29.99,Everyone,Medical
4264,Golfshot Plus: Golf GPS,SPORTS,4.10,3387,25.00,50000,Paid,29.99,Everyone,Sports
3241,"Muscle Premium - Human Anatomy, Kinesiology, B...",MEDICAL,4.20,168,25.00,10000,Paid,24.99,Everyone,Medical
4772,Human Anatomy Atlas 2018: Complete 3D Human Body,MEDICAL,4.50,2921,25.00,100000,Paid,24.99,Everyone,Medical


### Highest Grossing Paid Apps (ballpark estimate)

In [37]:
df_price['Revenue_estimate'] = df_price.Installs * df_price.Price  # also: df_price['Revenue_Estimate'] = df_price.Installs.mul(df_price.Price)
df_price.nlargest(10, 'Revenue_estimate')

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_estimate
9220,Minecraft,FAMILY,4.50,2376564,19.00,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,"69,900,000.00"
8825,Hitman Sniper,GAME,4.60,408292,29.00,10000000,Paid,0.99,Mature 17+,Action,"9,900,000.00"
7151,Grand Theft Auto: San Andreas,GAME,4.40,348962,26.00,1000000,Paid,6.99,Mature 17+,Action,"6,990,000.00"
7477,Facetune - For Free,PHOTOGRAPHY,4.40,49553,48.00,1000000,Paid,5.99,Everyone,Photography,"5,990,000.00"
7977,Sleep as Android Unlock,LIFESTYLE,4.50,23966,0.85,1000000,Paid,5.99,Everyone,Lifestyle,"5,990,000.00"
6594,DraStic DS Emulator,GAME,4.60,87766,12.00,1000000,Paid,4.99,Everyone,Action,"4,990,000.00"
6082,Weather Live,WEATHER,4.50,76593,4.75,500000,Paid,5.99,Everyone,Weather,"2,995,000.00"
6746,Card Wars - Adventure Time,FAMILY,4.30,129603,23.00,1000000,Paid,2.99,Everyone 10+,Card;Action & Adventure,"2,990,000.00"
6856,Threema,COMMUNICATION,4.50,51110,3.50,1000000,Paid,2.99,Everyone,Communication,"2,990,000.00"
7044,Tasker,TOOLS,4.60,43045,3.40,1000000,Paid,2.99,Everyone,Tools,"2,990,000.00"


# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [38]:
df_price.Category.nunique()  # counts unique cathegories 


33

In [39]:
top_10_apps_categories = df_price.Category.value_counts()[:10] # calculates amout of apps per category and picks the top 10
top_10_apps_categories

FAMILY             1606
GAME                910
TOOLS               719
PRODUCTIVITY        301
PERSONALIZATION     298
LIFESTYLE           297
FINANCE             296
MEDICAL             292
PHOTOGRAPHY         263
BUSINESS            262
Name: Category, dtype: int64

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [40]:
bar = px.bar(x=top_10_apps_categories.index, y=top_10_apps_categories.values, width=900)
bar.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [41]:
category_installs = df_price.groupby('Category').agg({'Installs': pd.Series.sum})  # groups the df_price by Category column then sums up the number of installs
category_installs.sort_values('Installs', ascending=True, inplace=True)  # 
category_installs[-10:]

,Installs
Category,
NEWS_AND_MAGAZINES,2369110650
TRAVEL_AND_LOCAL,2894859300
VIDEO_PLAYERS,3916897200
FAMILY,4437554490
PHOTOGRAPHY,4649143130
SOCIAL,5487841475
PRODUCTIVITY,5788070180
TOOLS,8099724500
COMMUNICATION,11039241530


In [42]:
h_bar = px.bar(x = category_installs.Installs, y = category_installs.index, orientation = 'h', title='Category Popularity', width=900)
h_bar.update_layout(xaxis_title='Number of downloads', yaxis_title='Category')
h_bar.show()

### Category Concentration - Downloads vs. Competition

**Challenge**: 
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<!-- <img src=https://imgur.com/uQRSlXi.png width="350"> -->

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this. 

<!-- <img src=https://imgur.com/cHsqh6a.png> -->

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log') 

In [43]:
category_apps_installs = df_price.groupby('Category').agg({'App':pd.Series.count, 'Installs': pd.Series.sum})  # also possible create another df with app count alone and merge it together
category_apps_installs.sort_values('Installs', ascending=False, inplace=True)
category_apps_installs[:10]

,App,Installs
Category,,
GAME,910,13858762717
COMMUNICATION,257,11039241530
TOOLS,719,8099724500
PRODUCTIVITY,301,5788070180
SOCIAL,203,5487841475
PHOTOGRAPHY,263,4649143130
FAMILY,1606,4437554490
VIDEO_PLAYERS,148,3916897200
TRAVEL_AND_LOCAL,187,2894859300


In [44]:
# scatter_plot = px.scatter(y=category_apps_installs.Installs, x=category_apps_installs.App, size=category_apps_installs.App, color=category_apps_installs.Installs, hover_name=category_apps_installs.index, width=900, height=600, title='Category Concentration')
scatter_plot = px.scatter(category_apps_installs, x='App', y='Installs', size='App', color='Installs', hover_name=category_apps_installs.index, width=900, height=500, title='Category Concentration')
scatter_plot.update_layout(xaxis_title='Number of Apps = More concentrated', yaxis_title='Installs', yaxis=dict(type='log'))  #, xaxis=dict(type='log') also looks great
scatter_plot.show()

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). 


In [45]:
df_price.Genres.value_counts()

Tools                                718
Entertainment                        467
Education                            429
Productivity                         301
Personalization                      298
                                    ... 
Adventure;Brain Games                  1
Travel & Local;Action & Adventure      1
Art & Design;Pretend Play              1
Music & Audio;Music & Video            1
Lifestyle;Pretend Play                 1
Name: Genres, Length: 114, dtype: int64

In [46]:
genres = df_price.Genres.str.split(';', expand=True)  # splits string into separate columns
genres_count = genres.stack().value_counts() # stack the separate columns in the rows then counts the
genres_count

Tools                      719
Education                  587
Entertainment              498
Action                     304
Productivity               301
Personalization            298
Lifestyle                  298
Finance                    296
Medical                    292
Sports                     270
Photography                263
Business                   262
Communication              258
Health & Fitness           245
Casual                     216
News & Magazines           204
Social                     203
Simulation                 200
Travel & Local             187
Arcade                     185
Shopping                   180
Books & Reference          171
Video Players & Editors    150
Dating                     134
Puzzle                     124
Maps & Navigation          118
Role Playing               111
Racing                     103
Action & Adventure          96
Strategy                    95
Food & Drink                94
Educational                 93
Adventur

In [47]:
len(genres_count)

53

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data? 

<!-- <img src=https://imgur.com/DbcoQli.png width=400> -->

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/). 

* Find a way to set the colour scale using the color_continuous_scale parameter. 
* Find a way to make the color axis disappear by using coloraxis_showscale. 

In [48]:
genres_count_bar = px.bar(genres_count[:15], width=900, color=genres_count[:15].values, color_continuous_scale='Agsunset', title='Top Genres')
genres_count_bar.update_layout(xaxis_title='Genre', yaxis_title='Number of apps', coloraxis_showscale=False)
genres_count_bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [49]:
df_price.Type.value_counts()

Free    7595
Paid     589
Name: Type, dtype: int64

In [50]:
df_free_vs_paid = df_price.groupby(["Category", "Type"]).agg({'App': pd.Series.count})  
df_free_vs_paid

App
Category          Type     
ART_AND_DESIGN    Free   58
                  Paid    3
AUTO_AND_VEHICLES Free   72
                  Paid    1
BEAUTY            Free   42
...                     ...
TRAVEL_AND_LOCAL  Paid    8
VIDEO_PLAYERS     Free  144
                  Paid    4
WEATHER           Free   65
                  Paid    7

[61 rows x 1 columns]

In [51]:
df_free_vs_paid = df_price.groupby(["Category", "Type"], as_index=False).agg({'App': pd.Series.count})  # as_index=False doesnt push category into index, instead makes a column - also it doesnt make it look stacked
df_free_vs_paid

,Category,Type,App
0,ART_AND_DESIGN,Free,58
1,ART_AND_DESIGN,Paid,3
2,AUTO_AND_VEHICLES,Free,72
3,AUTO_AND_VEHICLES,Paid,1
4,BEAUTY,Free,42
...,...,...,...
56,TRAVEL_AND_LOCAL,Paid,8
57,VIDEO_PLAYERS,Free,144
58,VIDEO_PLAYERS,Paid,4
59,WEATHER,Free,65


**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart: 

<!-- <img src=https://imgur.com/LE0XCxA.png> -->

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category. 

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value). 

In [52]:
# creating a separate df with types Free and Paid for my solution
df_free = df_free_vs_paid[df_free_vs_paid['Type']=='Free'] #  .sort_values(by='App', ascending=False)
df_paid = df_free_vs_paid[df_free_vs_paid['Type']=='Paid']

In [53]:
# My solution
free_paid_chart = go.Figure()

free_paid_chart.add_trace(go.Bar(
    x=df_free.Category,
    y=df_free.App,
    name= 'Free',
))

free_paid_chart.add_trace(go.Bar(
    x=df_paid.Category,
    y=df_paid.App,
    name= 'Paid',
))

free_paid_chart.update_layout(title='Free vs Paid Apps by Category', xaxis_title='Category', yaxis_title='Number of apps', width=1200, yaxis=dict(type='log'))
free_paid_chart.update_xaxes(categoryorder='total descending')

free_paid_chart.show()

In [54]:
# Angela's solution
g_bar = px.bar(
    df_free_vs_paid,
    x='Category',
    y='App',
    barmode='group',
    title='Free vs Paid Apps by Category',
    color='Type'  # this changed the graph layout - from going on top of eachother to next to eachother, also color
)

g_bar.update_layout(
    xaxis={'categoryorder':'total descending'},  # this is also intresting
    xaxis_title='Category', 
    yaxis_title='Number of apps', 
    width=1200, 
    yaxis=dict(type='log'),
)


g_bar.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart. 

<!-- <img src=https://imgur.com/uVsECT3.png> -->


In [55]:
# # This wasn't even necessary
# df_free_vs_paid_installs = df_price.groupby(["Category", "Type"], as_index=False).agg({'Installs': pd.Series.sum})
# df_free_vs_paid_installs

In [56]:
box_free_vs_paid_installs= px.box(df_price, x='Type', y='Installs', points='all', color='Type', width=900, notched=True, title='How Many Downloads are Paid Apps Giving Up?')
box_free_vs_paid_installs.update_layout(yaxis=dict(type='log'))
box_free_vs_paid_installs.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below: 

<!-- <img src=https://imgur.com/v4CiNqX.png> -->

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories. 

In [57]:
paid_apps = df_price[df_price.Type=='Paid']

In [58]:
box_revenue_by_app = px.box(paid_apps, x='Category', y='Revenue_estimate', title='How Much Can Paid Apps Earn?', width=1200)
box_revenue_by_app.update_layout(yaxis=dict(type='log'), xaxis_title='Paid App Ballpark Revenue', yaxis_title='Category', xaxis={'categoryorder':'min ascending'})
box_revenue_by_app

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [79]:
paid_apps.Price.median()

2.99

In [80]:
paid_apps.groupby('Category', as_index=False).agg({'Price': pd.Series.median}).sort_values(by='Price')

,Category,Price
22,SOCIAL,0.99
18,PERSONALIZATION,1.49
26,VIDEO_PLAYERS,1.74
0,ART_AND_DESIGN,1.99
1,AUTO_AND_VEHICLES,1.99
16,NEWS_AND_MAGAZINES,1.99
4,COMMUNICATION,2.70
21,SHOPPING,2.74
24,TOOLS,2.99
23,SPORTS,2.99


In [81]:
box_median = px.box(paid_apps, x='Category', y='Price', width=1200, title='Price per category')
box_median.update_layout(yaxis=dict(type='log'), xaxis={'categoryorder':'max descending'})
box_median.show()

# Summary

* Pull a random sample from a DataFrame using `.sample()`

* How to find duplicate entries with `.duplicated()` and `.drop_duplicates()`

* How to convert string and object data types into numbers with `.to_numeric()`

* How to use plotly to generate beautiful pie, donut, and bar charts as well as box and scatter plots